<a href="https://colab.research.google.com/github/yookkk1/stats/blob/master/quickstarts/Prompting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>